In [ ]:
# トピックのユーザー入力を求める
topic = input("研究トピックを入力してください: ")

研究トピックを入力してください: 熱・風以外の方法で髪の毛を乾かす技術のトレンドを教えて


# WEB-学術研究 (STORM)

[STORM](https://arxiv.org/abs/2402.14207) は、Shao氏らによって設計された研究アシスタントで、より充実した記事生成のために「アウトライン駆動型RAG」の考えを拡張したものです。

STORMは、ユーザーが提供したトピックについてWikipediaスタイルの記事を生成するように設計されています。より体系的で包括的な記事を作成するために、以下の2つの主要な洞察を適用します：

1. 類似トピックを検索してアウトライン（計画）を作成することで、カバレッジを向上させる
2. 多視点的で検索に基づく会話シミュレーションにより、参考文献数と情報密度を高める

制御フローは以下の図のようになります。

STORMには以下の主要なステージがあります：

1. 初期アウトラインの生成 + 関連テーマの調査
2. 異なる視点の特定
3. 「専門家へのインタビュー」（ロールプレイングLLM）
4. アウトラインの改善（参考文献を使用）
5. セクションの執筆、そして記事の執筆

専門家インタビューのステージは、ロールプレイする記事執筆者と研究専門家の間で行われます。「専門家」は外部知識を検索し、的確な質問に回答することができ、引用元をベクトルストアに保存して、後の改善段階で完全な記事を統合できるようにします。

（潜在的に）無限の研究範囲を制限するために設定できるハイパーパラメータがいくつかあります：

N: 調査/使用する視点の数（ステップ2->3）
M: ステップでの最大会話ターン数（ステップ3）

## セットアップ

まず、必要なパッケージをインストールしてAPIキーを設定します

In [ ]:
%%capture --no-stderr
%pip install -U langchain_community langchain_openai langchain_fireworks langgraph wikipedia duckduckgo-search tavily-python

In [ ]:
import getpass
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI

def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")

# OpenAlexメールアドレスの入力を求める
openalex_email = getpass.getpass("OpenAlexメールアドレスを入力してください: ")
os.environ["OPENALEX_EMAIL"] = openalex_email

# API認証情報の入力を求める
api_key = getpass.getpass("OpenAI APIキーを入力してください: ")
base_url = getpass.getpass("ベースURLを入力してください: ")

# クライアントの初期化
client = OpenAI(base_url=base_url, api_key=api_key)

OpenAlexメールアドレスを入力してください: ··········
OpenAI APIキーを入力してください: ··········
ベースURLを入力してください: ··········


### LLMの選択

In [ ]:
from langchain_openai import ChatOpenAI

model = "openai.gpt-4o-mini"
llm = ChatOpenAI(openai_api_base=base_url, openai_api_key=api_key, model=model)

## 初期アウトラインの生成

多くのトピックについて、LLMは重要な関連トピックの初期アイデアを持っています。研究後に改良される
初期アウトラインを生成することができます。

In [ ]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたはWikipediaの執筆者です。ユーザーが提供したトピックについてのWikipediaページの概要を作成してください。包括的かつ具体的に書いてください。",
        ),
        ("user", "{topic}"),
    ]
)


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="小節のタイトル")
    description: str = Field(..., title="小節の内容")

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.description}".strip()


class Section(BaseModel):
    section_title: str = Field(..., title="節のタイトル")
    description: str = Field(..., title="節の内容")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Wikipediaページの各小節のタイトルと説明",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"### {subsection.subsection_title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{self.description}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Wikipediaページのタイトル")
    sections: List[Section] = Field(
        default_factory=list,
        title="Wikipediaページの各節のタイトルと説明",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


generate_outline_direct = direct_gen_outline_prompt | llm.with_structured_output(
    Outline
)

In [ ]:
initial_outline = generate_outline_direct.invoke({"topic": topic})

print(initial_outline.as_str)

# 熱・風以外の髪の毛を乾かす技術

## 概要

髪の毛を乾かす方法は、主に熱と風を利用するものが一般的ですが、近年ではそれ以外の干渉技術が注目されています。これらの新しい技術は、髪へのダメージを減少させ、より髪の健康を考慮したアプローチを提供します。特に低温や無風での乾燥方法が注目されています。これらは特に敏感な髪質やダメージが気になる人々に適しています。さらに、環境への配慮から省エネルギーでの乾燥方法にも需要が高まっています。　実際にこれらの技術は、日常生活における使い方や、商品展開にも影響を与えています。.

### 水分蒸発を促進する技術

特殊な素材を使用したタオルやマイクロファイバーで、髪の水分を素早く吸収し、化学的に髪を乾かす技術が徐々に普及しています。これにより、熱や風を使わなくても髪を効果的に乾かすことが可能となります。

### しずく軽減技術

新しい乾燥機能を持つヘアケア製品のいくつかは、髪に残ったしずくを軽減する技術を使っています。これにより、毛髪の表面から水分が早く除去され、乾燥時間を短縮する効果があります。

### 超音波技術

超音波波動を利用して、髪の内部に微細な気泡を生成し、その振動によって水分を蒸発させる技術が研究されています。これにより、髪に優しい方法でスピーディーに乾かすことが期待されています。

### エコ乾燥法

環境意識の高まりに伴い、電力を使わずに自然の流れや温度を利用して乾燥する技術も注目されています。例えば、風通しの良い場所で自然乾燥させる方法に加え、新しいデザインのドライシャンプーなども取り入れられています。


## トピックの展開

言語モデルはパラメータ内にWikipedia的な知識を保持していますが、検索エンジンを使用して関連する最新の情報を取り入れることでより良い結果が得られます。

Wikipediaから取得した関連トピックのリストを生成することから検索を開始します。

In [ ]:
gen_related_topics_prompt = ChatPromptTemplate.from_template(
    """以下のトピックについてWikipediaページを執筆しています。関連する主題のWikipediaページを特定し、推奨してください。このトピックに一般的に関連する興味深い側面についての洞察を提供する例や、類似トピックのWikipediaページに含まれる典型的な内容と構造を理解するのに役立つ例を探しています。
できるだけ多くの関連主題とURLを列挙してください。
対象トピック: {topic}
"""
)


class RelatedSubjects(BaseModel):
    topics: List[str] = Field(
        description="背景調査のための関連主題の包括的なリスト",
    )


expand_chain = gen_related_topics_prompt | llm.with_structured_output(
    RelatedSubjects
)

In [ ]:
related_subjects = await expand_chain.ainvoke({"topic": topic})
related_subjects

RelatedSubjects(topics=['ドライヤー', '髪の毛の乾燥方法', '髪の毛のスタイリング技術', 'エコフレンドリーな美容技術', '髪の毛ケア製品のトレンド', 'ヘアサロンのサービス', '自宅での髪の毛ケア', '乾燥機の技術革新', '日本における髪の美容文化', '温風以外の髪の乾燥技術', 'ヘアケアの歴史', '空気乾燥技術'])

## 視点の生成

これらの関連テーマから、異なる背景と所属を持つ「専門家」としてWikipedia編集者の代表を選択することができます。
これにより検索プロセスを分散させ、より総合的な最終レポートの作成を促進します。

In [ ]:
class Editor(BaseModel):
    affiliation: str = Field(
        description="編集者の主な所属",
    )
    name: str = Field(
        description="編集者の名前", pattern=r"^[a-zA-Z0-9_-]{1,64}$"
    )
    role: str = Field(
        description="トピックに関連する編集者の役割",
    )
    description: str = Field(
        description="編集者の焦点、関心事、動機の説明",
    )

    @property
    def persona(self) -> str:
        return f"名前: {self.name}\n役割: {self.role}\n所属: {self.affiliation}\n説明: {self.description}\n"


class Perspectives(BaseModel):
    editors: List[Editor] = Field(
        description="編集者とその役割、所属の包括的なリスト",
        # 編集者の数をM以下に制限するpydanticバリデーション/制限を追加
    )


gen_perspectives_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """トピックについて包括的な記事を作成するために協力して作業する、多様で個性的なWikipedia編集者グループを選定する必要があります。各編集者は、このトピックに関連する異なる視点、役割、所属を代表します。\

    着想を得るために、関連トピックの他のWikipediaページを参考にすることができます。各編集者について、何に焦点を当てるかの説明を追加してください。

    着想を得るための関連トピックのWikiページの概要：

    {examples}""",
        ),
        ("user", "対象トピック: {topic}"),
    ]
)

gen_perspectives_chain = (
    gen_perspectives_prompt
    | ChatOpenAI(
        openai_api_base=base_url,
        openai_api_key=api_key,
        model="openai.gpt-3.5-turbo"
    ).with_structured_output(
        Perspectives,
        method="function_calling"
    )
)

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain as as_runnable

wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)


def format_doc(doc, max_length=1000):
    related = "- ".join(doc.metadata["categories"])
    return f"### {doc.metadata['title']}\n\n概要: {doc.page_content}\n\n関連\n{related}"[
        :max_length
        ]


def format_docs(docs):
    return "\n\n".join(format_doc(doc) for doc in docs)


@as_runnable
async def survey_subjects(topic: str):
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})

In [ ]:
perspectives = await survey_subjects.ainvoke(topic)

In [ ]:
perspectives.dict()

<ipython-input-12-b975437d5130>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  perspectives.dict()


{'editors': [{'affiliation': 'Hair Salon Association',
   'name': 'MiaS',
   'role': 'Hair Stylist',
   'description': 'MiaSは専門家の美容師であり、熱・風以外の方法で髪の毛を乾かす技術の最新トレンドや効果的な方法に関心があります。彼女は常に新しいヘアケア技術に興味を持っており、その情報を共有することで他の美容師や顧客に役立てています。'},
  {'affiliation': 'Fashion Magazine',
   'name': 'AveryT',
   'role': 'Fashion Journalist',
   'description': 'AveryTはファッション雑誌のジャーナリストで、最新の美容トレンドやヘアケアに関する記事を執筆しています。彼女は熱・風以外の髪の毛を乾かす技術についての情報収集とトレンド分析に情熱を持っています。'},
  {'affiliation': 'Beauty Tech Startup',
   'name': 'EthanH',
   'role': 'Tech Innovator',
   'description': 'EthanHは美容テックスタートアップのイノベーターであり、新しい技術やデバイスを開発しています。彼は熱・風以外の髪の毛を乾かす技術に革新的なアプローチをもたらすことに興味があります。'}]}

## 専門家との対話

各Wikipedia編集者は、上記の視点を使用してロールプレイを行うよう準備されています。検索エンジンにアクセスできる第二の「専門家」に一連の質問を行います。これにより、改良されたアウトラインと更新された参考文献インデックスを生成するためのコンテンツを生成します。


### インタビューの状態

会話は循環的なので、独自のグラフ内に構築します。この状態にはメッセージ、参考文献、エディター（独自の「ペルソナ」を持つ）が含まれ、これらの会話を並列化しやすくします。

In [ ]:
from typing import Annotated

from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph, START


def add_messages(left, right):
    if not isinstance(left, list):
        left = [left]
    if not isinstance(right, list):
        right = [right]
    return left + right


def update_references(references, new_references):
    if not references:
        references = {}
    references.update(new_references)
    return references


def update_editor(editor, new_editor):
    # 開始時にのみ設定可能
    if not editor:
        return new_editor
    return editor


class InterviewState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    references: Annotated[Optional[dict], update_references]
    editor: Annotated[Optional[Editor], update_editor]

#### 対話の役割

グラフには2人の参加者がいます：割り当てられた役割に基づいて質問をするWikipedia編集者（`generate_question`）と、検索エンジンを使用して可能な限り正確に質問に回答する専門家（`gen_answer_chain`）です。

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import MessagesPlaceholder

gen_qn_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """あなたは経験豊富なWikipedia編集者で、特定のページを編集したいと考えています。
Wikipedia編集者としての立場に加えて、トピックを研究する際の特定の焦点があります。
現在、専門家と情報を得るためにチャットをしています。有用な情報を得るために、良い質問をしてください。
質問がなくなったら、「ご協力ありがとうございました！」と言って会話を終了してください。
質問は一度に一つずつ行い、以前に尋ねた質問は繰り返さないでください。
質問は執筆したいトピックに関連したものにしてください。
包括的で好奇心を持って、専門家からできるだけ多くのユニークな洞察を得るようにしてください。\
あなたの特定の視点を忠実に守ってください：
{persona}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


def tag_with_name(ai_message: AIMessage, name: str):
    ai_message.name = name
    return ai_message


def swap_roles(state: InterviewState, name: str):
    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    return {"messages": converted}


@as_runnable
async def generate_question(state: InterviewState):
    editor = state["editor"]
    gn_chain = (
        RunnableLambda(swap_roles).bind(name=editor.name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | llm
        | RunnableLambda(tag_with_name).bind(name=editor.name)
    )
    result = await gn_chain.ainvoke(state)
    return {"messages": [result]}

In [ ]:
messages = [
    HumanMessage(f"では、{topic}について記事を書いているとおっしゃっていましたか？")
]
question = await generate_question.ainvoke(
    {
        "editor": perspectives.editors[0],
        "messages": messages,
    }
)

question["messages"][0].content

'はい、正確にその通りです。熱・風以外の方法で髪の毛を乾かす技術のトレンドに関心があります。その分野における最新の技術やトレンドについて詳しくお教えいただけますか？特に注目すべき製品や技術があれば教えてください。'

#### 質問への回答

`gen_answer_chain`は最初に編集者の質問に答えるためのクエリ（クエリ拡張）を生成し、その後引用付きで回答します。

In [ ]:
class Queries(BaseModel):
    queries: List[str] = Field(
        description="ユーザーの質問に答えるための検索エンジンクエリの包括的なリスト",
    )
    web_queries: List[str] = Field(
        description="ユーザーの質問に答えるためのウェブ検索エンジンクエリの包括的なリスト",
    )
    scholarly_queries: List[str] = Field(
        description="ユーザーの質問に答えるための学術検索エンジンクエリの包括的なリスト",
    )

    @classmethod
    def create(cls, web_queries: List[str], scholarly_queries: List[str]):
        return cls(
            queries=web_queries + scholarly_queries,
            web_queries=web_queries,
            scholarly_queries=scholarly_queries
        )

# Model classes for each query type
class WebQueries(BaseModel):
    queries: List[str] = Field(description="ウェブ検索クエリ")

class ScholarlyQueries(BaseModel):
    queries: List[str] = Field(description="学術検索クエリ")

# Simple, focused prompts for each type of search
gen_web_queries_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは役立つリサーチアシスタントです。ユーザーの質問に答えるために検索エンジンにクエリを実行してください。",
    ),
    MessagesPlaceholder(variable_name="messages", optional=True),
]) | llm.with_structured_output(WebQueries)

gen_scholarly_queries_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは役立つリサーチアシスタントです。ユーザーの質問に答えるためにOpenAlex検索エンジンにクエリを実行してください。",
    ),
    MessagesPlaceholder(variable_name="messages", optional=True),
]) | llm.with_structured_output(ScholarlyQueries)

# Combine the chains
from langchain.schema.runnable import RunnableParallel

def combine_results(results):
    return {
        "raw": None,
        "parsed": Queries.create(
            web_queries=results["web"].queries,
            scholarly_queries=results["scholarly"].queries
        )
    }

gen_queries_chain = (
    RunnableParallel({
        "web": gen_web_queries_prompt,
        "scholarly": gen_scholarly_queries_prompt
    })
    | combine_results
)

In [ ]:
queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
print("ウェブ検索クエリ", queries["parsed"].web_queries)
print("学術検索クエリ", queries["parsed"].scholarly_queries)

ウェブ検索クエリ ['最新の髪の毛を乾かす技術', '非熱乾燥髪の毛 製品 トレンド 2023', '髪の毛 乾かす 技術 革新', '熱以外の方法 髪乾燥 最新', 'ノンドライヤー 髪の毛の乾かし方', '水分保持する髪の毛 乾燥方法 または製品', 'ミスト技術 髪の毛 乾かし方 2023', '外部データ 髪 乾燥 風以外 トレンド']
学術検索クエリ ['最新の髪の毛乾燥技術', '熱以外の髪の毛乾燥方法', '風以外の髪の毛乾燥技術のトレンド', '髪の毛乾燥のための革新的な製品', '髪の毛乾燥における技術革新', '静電気を利用した髪の毛の乾燥技術', '髪の毛を乾かす新しいプロセス', '空気圧を活用したヘアドライ技術']


In [ ]:
class AnswerWithCitations(BaseModel):
    answer: str = Field(
        description="ユーザーの質問に対する包括的な回答（引用付き）。",
    )
    cited_urls: List[str] = Field(
        description="回答内で引用されているURLのリスト。",
    )

    @property
    def as_str(self) -> str:
        return f"{self.answer}\n\nCitations:\n\n" + "\n".join(
            f"[{i+1}]: {url}" for i, url in enumerate(self.cited_urls)
        )


gen_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """あなたは情報を効果的に活用できる専門家です。あなたは、自分の知っているトピックについてWikipediaに記事を書きたいと考えているライターと会話をしています。関連情報を収集済みで、その情報を活用して回答を作成しようとしています。

回答はできる限り情報量を多くし、収集した情報で裏付けられるようにしてください。
すべての文は信頼できる情報源の引用（脚注形式）によって裏付けられている必要があります。回答の後にURLを再掲する形で引用を示してください。""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

gen_answer_chain = gen_answer_prompt | llm.with_structured_output(
    AnswerWithCitations, include_raw=True
).with_config(run_name="GenerateAnswer")


In [ ]:
import os
import requests
from typing import List, Dict
from langchain_core.tools import tool

class OpenAlexSearchAPIWrapper:
    """OpenAlex APIのラッパー。"""
    def __init__(self, openalex_email: str = None):
        self.openalex_email = openalex_email or os.getenv("OPENALEX_EMAIL")

    def _text(self, query: str) -> List[Dict]:
        headers = {'User-Agent': f'mailto:{self.openalex_email}'}
        url = "https://api.openalex.org/works"
        params = {
            "search": str(query),
            "per-page": 25,  # Increased to ensure we get enough results with abstracts
            "page": 1,
            "sort": "relevance_score:desc",
            "select": "id,display_name,publication_year,abstract_inverted_index,doi,primary_location,cited_by_count"
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            results = []
            for work in data.get('results', []):
                if not work:
                    continue

                # Skip if there's no abstract
                if not work.get('abstract_inverted_index'):
                    continue

                # Process abstract
                abstract_index = work['abstract_inverted_index']
                word_positions = {pos: word for word, positions in abstract_index.items() for pos in positions}
                abstract = " ".join(word_positions[i] for i in sorted(word_positions.keys()))

                # Skip if abstract is empty after processing
                if not abstract.strip():
                    continue

                url = None
                if work.get('doi'):
                    url = f"https://doi.org/{work['doi'].replace('https://doi.org/', '')}"
                elif work.get('primary_location', {}).get('doi'):
                    url = f"https://doi.org/{work['primary_location']['doi']}"
                elif work.get('id'):
                    work_id = work['id'].replace('https://openalex.org/', '')
                    url = f"https://openalex.org/{work_id}"

                publication_year = work.get('publication_year', 'Year not available')
                cited_by_count = work.get('cited_by_count', 0)

                if url:
                    results.append({
                        "content": f"Title: {work.get('display_name', '')}\nYear: {publication_year}\nCitations: {cited_by_count}\nAbstract: {abstract}",
                        "url": url
                    })

                # Break once we have 5 results with abstracts
                if len(results) == 5:
                    break

            return results
        return []

# 検索エンジンの初期化
scholarly_search_engine = OpenAlexSearchAPIWrapper()

@tool
async def scholarly_search_engine(query: str):
    """学術研究のための検索エンジン。"""
    results = OpenAlexSearchAPIWrapper()._text(query)
    return results

In [ ]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

# DDG
web_search_engine = DuckDuckGoSearchAPIWrapper()


@tool
async def web_search_engine(query: str):
    """Search engine to the internet."""
    results = DuckDuckGoSearchAPIWrapper()._ddgs_text(query)
    return [{"content": r["body"], "url": r["href"]} for r in results]

In [ ]:
import json
from typing import Optional
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import AIMessage, FunctionMessage, SystemMessage

async def gen_scholarly_answer(
    state: InterviewState,
    queries,
    config: Optional[RunnableConfig] = None,
    name: str = "Scholarly_Expert",
    max_str_len: int = 15000,
):
    print("Starting scholarly search...")
    swapped_state = swap_roles(state, name)

    # Execute scholarly search
    query_results = await scholarly_search_engine.abatch(
        queries.scholarly_queries, config, return_exceptions=True
    )

    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]

    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }

    # Add search results to state for the answer generation
    search_msg = SystemMessage(content=json.dumps(all_query_results)[:max_str_len])
    swapped_state["messages"].append(search_msg)

    # Generate the answer using the collected information
    generated = await gen_answer_chain.ainvoke(swapped_state)
    cited_urls = set(generated["parsed"].cited_urls)
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    formatted_content = generated["parsed"].as_str

    return {
        "content": formatted_content,
        "references": cited_references
    }

async def gen_web_answer(
    state: InterviewState,
    queries,
    config: Optional[RunnableConfig] = None,
    name: str = "Web_Expert",
    max_str_len: int = 15000,
):
    print("Starting web search...")
    swapped_state = swap_roles(state, name)

    query_results = await web_search_engine.abatch(
        queries.web_queries, config, return_exceptions=True
    )

    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]

    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }

    # Add search results to state for the answer generation
    search_msg = SystemMessage(content=json.dumps(all_query_results)[:max_str_len])
    swapped_state["messages"].append(search_msg)

    # Generate the answer using the collected information
    generated = await gen_answer_chain.ainvoke(swapped_state)
    cited_urls = set(generated["parsed"].cited_urls)
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    formatted_content = generated["parsed"].as_str

    return {
        "content": formatted_content,
        "references": cited_references
    }

async def gen_answer(
    state: InterviewState,
    config: Optional[RunnableConfig] = None,
    max_str_len: int = 15000,
):
    # Generate queries once
    queries = (await gen_queries_chain.ainvoke(state))["parsed"]

    # Get both scholarly and web answers using the same queries
    scholarly_result = await gen_scholarly_answer(state, queries, config)
    web_result = await gen_web_answer(state, queries, config)

    # Combine both perspectives into a single message
    combined_content = (
        "[Scholarly Works Perspective]\n" +
        scholarly_result["content"] +
        "\n\n---\n\n" +
        "[Web Perspective]\n" +
        web_result["content"]
    )

    # Create single message with all citations
    combined_message = AIMessage(
        name="Combined_Expert",
        content=combined_content
    )

    return {
        "messages": [combined_message],
        "references": {**scholarly_result["references"], **web_result["references"]}
    }

In [ ]:
example_answer = await gen_answer(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
example_answer["messages"][-1].content

Starting scholarly search...
Starting web search...


'[Scholarly Works Perspective]\n近年、髪の乾燥技術におけるトレンドとしては、熱や風以外の方法が注目を集めています。これには主に超音波乾燥、遠赤外線乾燥、さらには自然乾燥の技術が含まれます。これらの技術は、髪の健康を守りつつ、効率よく乾燥させることを目的としています。\n\n### 1. 超音波乾燥\n超音波技術を用いた乾燥方法は、水分子を振動させることで熱を発生させ、髪の内部から外部に向けて水分を蒸発させるというものです。この技術は、髪へのダメージを減少させることができるため、特にダメージヘアの人々にとって魅力的です【1】。\n\n### 2. 遠赤外線乾燥\n遠赤外線は、物質を加熱することなく、その内部から直接水分を蒸発させる能力を持っています。これにより、従来の乾燥方法に比べて、髪の毛を乾かす時間を短縮し、髪の質感を保持することが可能です。また、遠赤外線は抗菌作用もあるため、髪の健康を保つ助けにもなります【2】。\n\n### 3. 自然乾燥の技術\n自然乾燥も進化しており、特に最近は、髪の毛が自然に乾くプロセスを助けるための製品が市場に登場しています。これには、髪を包み込む自然素材のタオルや、特定のセラムやオイルが含まれます。それらは髪に水分を閉じ込めながら、乾燥を促進する効果があります【3】。\n\n### 注目すべき製品\n- **超音波ドライヤー**: 新しい超音波技術を搭載したドライヤーが登場し、髪のダメージを最小限に抑えることができます。\n- **遠赤外線ヘアドライヤー**: このタイプのドライヤーは、髪の内部から乾かすため、熱ダメージのリスクを減少させます。\n- **ライスウォーターエッセンス**: 日本の伝統的な髪の手入れ方法を基にしたこの製品は、髪をしっとりとさせる効果があり、最近特に人気を集めています【4】。\n\nこれらの技術や製品は、髪の仕上がりを保ちながら、より効果的に乾燥させる手助けをし、多くの人々が求める美しい髪の実現に寄与しています。\n\nCitations:\n\n[1]: https://doi.org/10.5650/jos1996.45.1133\n[2]: https://doi.org/10.5107/sccj.25.33\n[3]: https://doi.org/10.5107

#### インタビューグラフの構築

編集者と専門家を定義したので、それらをグラフに組み合わせることができます。

In [ ]:
max_num_turns = 5
from langgraph.pregel import RetryPolicy


def route_messages(state: InterviewState, name: str = "Subject_Matter_Expert"):
    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )
    if num_responses >= max_num_turns:
        return END
    last_question = messages[-2]
    if last_question.content.endswith("ご協力ありがとうございました！"):
        return END
    return "ask_question"


builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question, retry=RetryPolicy(max_attempts=5))
builder.add_node("answer_question", gen_answer, retry=RetryPolicy(max_attempts=5))
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.add_edge(START, "ask_question")
interview_graph = builder.compile(checkpointer=False).with_config(
    run_name="Conduct Interviews"
)

In [ ]:
final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {topic}?",
            name="Subject_Matter_Expert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name]["messages"])[:300])
final_step = step

<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


ask_question
--  [AIMessage(content='そのトピックはとても興味深いですね！最近、熱や風以外の方法で髪を乾かす技術において注目されている革新的な手法や製品はありますか？特に、髪の健康に配慮した方法に焦点を当てているものがあれば教えていただけると嬉しいです。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 352, 'total_tokens': 439, 'completion_tokens_details': 
Starting scholarly search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


Starting web search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


...

ask_question
--  [AIMessage(content='ご協力ありがとうございました！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 5602, 'total_tokens': 5608, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_predic
Starting scholarly search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


Starting web search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


answer_question
--  [AIMessage(content='[Scholarly Works Perspective]\n最近の髪の毛を乾かす技術や方法の発展は、美容師としての業務に多くの影響を及ぼしています。以下にその影響をまとめます。  \n\n1. **顧客ニーズの変化**: 現在の顧客は、髪の健康を重視する傾向が強まっており、熱や風以外で髪を乾かす技術の導入を求めています。美容師は、この変化に適応し、顧客の健康を守るために新しい技術や商品に精通する必要があります[[1](https://www.abc-store-japan.com/diary-detail/428)]。  \n\n2. **技術の進化に


In [ ]:
final_state = next(iter(final_step.values()))

## アウトラインの改善

STORMのこの段階で、様々な視点から多くの研究を行いました。これらの調査に基づいて元のアウトラインを改善する時が来ました。以下では、長いコンテキストウィンドウを持つLLMを使用してチェーンを作成し、元のアウトラインを更新します。

In [ ]:
refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """あなたはWikipedia編集者です。専門家や検索エンジンから情報を収集しました。これから、Wikipediaページの概要を改善します。
概要が包括的かつ具体的であることを確認してください。
執筆するトピック: {topic}
以前の概要:
{old_outline}""",
        ),
        (
            "user",
            "専門家との会話に基づいて概要を改善してください：\n\n会話内容：\n\n{conversations}\n\n改善されたWikipediaの概要を作成してください：",
        ),
    ]
)

refine_outline_chain = refine_outline_prompt | llm.with_structured_output(
    Outline
)

## 記事の生成

いよいよ完全な記事を生成する時が来ました。まず分割統治法を用いて、各セクションを個別のLLMが担当できるようにします。その後、長文形式のLLMを使用して完成した記事を改善します（各セクションが一貫性のない文体を使用している可能性があるため）。

#### 検索機能の作成

研究プロセスでは、最終的な記事執筆プロセスで検索したい可能性のある多数の参考文献が見つかります。

まず、検索機能を作成します：

In [ ]:
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model="openai.text-embedding-3-large")
reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]
# このサイズのデータにはベクトルストアは実際には必要ありません。
# 単なるnumpy行列でも良いですし、リクエスト間でドキュメントを
# 保存することもできます。
vectorstore = InMemoryVectorStore.from_documents(
    reference_docs,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever(k=3)

#### セクションの生成

インデックス化された文書を使用してセクションを生成できるようになりました。

In [ ]:
class SubSection(BaseModel):
    subsection_title: str = Field(..., title="小節のタイトル")
    content: str = Field(
        ...,
        title="小節の完全な内容。関連する箇所には [#] の形式で出典を含めてください。",
    )

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.content}".strip()


class WikiSection(BaseModel):
    section_title: str = Field(..., title="節のタイトル")
    content: str = Field(..., title="節の完全な内容")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Wikipediaページの各小節のタイトルと説明",
    )
    citations: List[str] = Field(default_factory=list)

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        citations = "\n".join([f" [{i}] {cit}" for i, cit in enumerate(self.citations)])
        return (
            f"## {self.section_title}\n\n{self.content}\n\n{subsections}".strip()
            + f"\n\n{citations}".strip()
        )


section_writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは熟練したWikipedia編集者です。以下の概要から割り当てられたWikiSectionを完成させてください：\n\n"
            "{outline}\n\n以下の参考文献を用いて出典を記載してください：\n\n<Documents>\n{docs}\n<Documents>",
        ),
        ("user", "{section}セクションのWikiSectionを完全に執筆してください。"),
    ]
)


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}


section_writer = (
    retrieve
    | section_writer_prompt
    | llm.with_structured_output(WikiSection)
)

#### 最終記事の生成

ここで、すべての引用を適切にグループ化し、一貫した文体を維持するように草稿を書き直すことができます。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """あなたは熟練したWikipedia編集者です。以下のセクション原稿を使用して{topic}に関する完全なWiki記事を執筆してください：\n\n
            {draft}\n\nWikipediaの形式ガイドラインに厳密に従ってください。""",
        ),
        (
            "user",
            """マークダウン形式で完全なWiki記事を執筆してください。脚注は「[1]」のような形式で表記し、フッターで重複を避けてください。フッターにはURLを含めてください。""",
        ),
    ]
)

writer = writer_prompt | llm | StrOutputParser()

## 最終フロー

全てを組み合わせる時が来ました。順序立てて6つの主要な段階があります：
.
1. 初期アウトライン + 視点の生成
2. 記事のコンテンツを拡張するために各視点との一括会話
3. 会話に基づいてアウトラインの改善
4. 会話から参考文献のインデックス作成
5. 記事の個別セクションの執筆
6. 最終的なwikiの執筆

この状態は各段階の出力を追跡します。

In [ ]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # 最終セクションの出力
    sections: List[WikiSection]
    article: str

In [ ]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"では、{topic}について記事を書いているとおっしゃっていましたか？",
                    name="Subject_Matter_Expert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # ここでは、インタビューを並列化するためにサブグラフを呼び出す。
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'{interview_state["editor"].name}との会話\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }

async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### グラフの作成

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder.add_node(name, node, retry=RetryPolicy(max_attempts=3))
    if i > 0:
        builder.add_edge(nodes[i - 1][0], name)

builder.add_edge(START, nodes[0][0])
builder.add_edge(nodes[-1][0], END)
storm = builder.compile(checkpointer=MemorySaver())

In [ ]:
config = {"configurable": {"thread_id": "my-thread"}}
async for step in storm.astream({"topic": topic}, config):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name])[:300])

init_research
--  {'topic': '熱・風以外の方法で髪の毛を乾かす技術のトレンドを教えて', 'outline': Outline(page_title='熱・風以外の髪の毛を乾かす技術', sections=[Section(section_title='はじめに', description='髪の毛を乾かす方法は、従来の熱や風を利用する方法に加え、最近ではさまざまな新しい技術が登場しています。これらは髪に優しく、ダメージを最小限に抑えることが目的とされています。特に、熱や風を使わない方法は、髪の健康を重視する消費者から注目されているトレンドです。特に環境に優しい製品や、効率的に水分を除去できる技術が求


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


Starting scholarly search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


Starting scholarly search...
Starting scholarly search...


...

Starting web search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


Starting web search...


<ipython-input-14-938fd75fcdcb>:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  message = HumanMessage(**message.dict(exclude={"type"}))


conduct_interviews
--  {'topic': '熱・風以外の方法で髪の毛を乾かす技術のトレンドを教えて', 'outline': Outline(page_title='熱・風以外の髪の毛を乾かす技術', sections=[Section(section_title='はじめに', description='髪の毛を乾かす方法は、従来の熱や風を利用する方法に加え、最近ではさまざまな新しい技術が登場しています。これらは髪に優しく、ダメージを最小限に抑えることが目的とされています。特に、熱や風を使わない方法は、髪の健康を重視する消費者から注目されているトレンドです。特に環境に優しい製品や、効率的に水分を除去できる技術が求
refine_outline
--  {'topic': '熱・風以外の方法で髪の毛を乾かす技術のトレンドを教えて', 'outline': Outline(page_title='熱・風以外の髪の毛を乾かす技術', sections=[Section(section_title='はじめに', description='従来の熱や風を利用する方法に加え、最近では熱・風以外で髪の毛を乾かす新しい技術が多数登場しています。これらの技術は、髪の健康を重視し、より優れた仕上がりと環境への配慮を実現することを目的としています。特に、消費者の間では、髪へのダメージを最小限に抑える選択肢が求められています。近年のトレンドには、超音波技術、マイ
index_references
--  {'topic': '熱・風以外の方法で髪の毛を乾かす技術のトレンドを教えて', 'outline': Outline(page_title='熱・風以外の髪の毛を乾かす技術', sections=[Section(section_title='はじめに', description='従来の熱や風を利用する方法に加え、最近では熱・風以外で髪の毛を乾かす新しい技術が多数登場しています。これらの技術は、髪の健康を重視し、より優れた仕上がりと環境への配慮を実現することを目的としています。特に、消費者の間では、髪へのダメージを最小限に抑える選択肢が求められています。近年のトレンドには、超音波技術、マイ
write_sections
--  {'topic': '熱・風以

In [ ]:
checkpoint = storm.get_state(config)
article = checkpoint.values["article"]

# Wikiのレンダリング

これで最終的なwikiページをレンダリングできます！

In [ ]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))

# 熱・風以外の髪の毛乾燥技術

### はじめに

従来の熱や風を利用する方法に加え、最近では熱・風以外で髪の毛を乾かす新しい技術が多数登場しています。これらの技術は、髪の健康を重視し、より優れた仕上がりと環境への配慮を実現することを目的としています。特に、消費者の間では、髪へのダメージを最小限に抑える選択肢が求められています。近年のトレンドには、超音波技術、マイクロミスト技術、イオン技術、ナノファイバー技術など様々な革新が含まれています。これにより、消費者は髪を効率的に乾かしながら、髪質を改善する製品を求めるようになっています。

### 主な技術

#### 超音波技術

超音波技術は、音波を利用して水分を細かく分散させる方法です。この技術により、髪の毛内部の水分が効果的に蒸発し、乾燥が促進されます。超音波による振動は、髪の毛に優しく、ダメージを抑えつつ効率的に乾燥することができます。

#### マイクロミスト技術

マイクロミスト技術は、非常に細かい水の粒子を細かく噴射するシステムです。これにより、髪の毛表面から均等に水分が蒸発し、乾燥時間を短縮します。この技術は、髪への負担を軽減しながら、仕上がりのツヤや手触りにも大きな影響を与えます。

#### イオン技術

イオン技術は、髪に正負のイオンを届けることで、静電気を除去し、髪の毛の乾燥を助ける方法です。イオンが髪の毛に潤いをもたらし、スタイリングの際のダメージを軽減します。近年では、イオン技術を搭載したドライヤーやスタイリングツールが多く登場しています。

#### ナノファイバー技術

ナノファイバー技術は、ナノレベルの繊維を用いて水分を効果的に吸収する技術です。この繊維は非常に高い吸水性を持ち、髪の毛表面から水分を迅速に取り除くことが可能です。ナノファイバー技術を利用したタオルやマントなどの製品が人気です。

### 結論

熱や風を使用せずに髪を乾かす新しい技術は、髪の健康を重視し、効率的かつ環境に優しい方法として注目を集めています。消費者はこれらの革新を通じて、髪に優しい乾燥方法を選択することで、より良いヘアケアを実現できるようになっています。

---

[1] https://www.demi.nicca.co.jp/media/904/  
[2] https://quickpcr.jp/contents/other/hairdry/  
[3] https://bihadashop.jp/entry/air-dry  
[4] https://www.livedoor.com/choice/haircare_tips-natural-drying/